In [6]:
import os
import sys
from pathlib import Path
import re
import jieba

In [2]:
# import shutil

In [3]:
# for path_item in SLR_wav_list:
#     wav_path = str(path_item)
#     shutil.move(wav_path,wav_path[0:-4]) 
 

## 准备词典

In [4]:
# !cd data && python ../tools/prep_dict.py lexicon.txt

In [5]:
# !./tools/prepare_dict.sh data

In [6]:
# !rm -rf gmm/data/dict/
# !mkdir gmm/data/dict/
# !cd data &&cp lexicon.txt silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt ../gmm/data/dict
# # !rm silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt

## SLR85生成wav.scp

In [7]:
SLR85_dir = "/home1/meichaoyang/Dataset/feats/SLR85/hifi/train"
SLR_wav_list=list(Path(SLR85_dir).rglob("*.wav"))

In [8]:
SLR85_dir = "/home1/meichaoyang/Dataset/feats/SLR85/hifi/dev"
SLR_wav_list=list(Path(SLR85_dir).rglob("*.wav")) + SLR_wav_list

In [9]:
SLR_wav_list.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [10]:
len(SLR_wav_list)

2377

In [11]:
with open('SLR_wav.scp', 'w') as f:
    for path_item in SLR_wav_list:
        wav_path = str(path_item)
        f.write(os.path.basename(os.path.splitext((wav_path))[0])+'\t'+wav_path+'\n')

### 并筛选一部分放入SLR_wav.scp.shuff文件中

In [12]:
# !bash tools/random_line.sh SLR_wav.scp 100 SLR_wav.scp.shuff
!cat SLR_wav.scp > SLR_wav.scp.shuff

> 并筛选一部分放入SLR_wav.scp.shuff文件中

## Aishell2数据处理

### 生成Aishell2的原始wav.scp

In [13]:
aishell_dir = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/train"
aishell_wav_list=list(Path(aishell_dir).rglob("*.wav"))
aishell_wav_list.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [14]:
aishell_scp_raw = {}
with open('Aishell_wav_raw.scp', 'w') as f:
    for path_item in aishell_wav_list:
        wav_path = str(path_item)
        aishell_scp_raw[os.path.basename(os.path.splitext((wav_path))[0])] = wav_path
        f.write(os.path.basename(os.path.splitext((wav_path))[0])+'\t'+wav_path+'\n')

### 对满足条件的标注进行筛选

In [15]:
pattern_utt = re.compile(r'/.*\.')
pattern_Eng = re.compile(u'[a-zA-Z\n]')

In [16]:
aishell_2_corp_raw = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/trans.txt"
corp_map = {}
with open(aishell_2_corp_raw, "r") as f:
    for line in f:
        data = line.split()
        if pattern_Eng.search(data[1]) != None: ##删除小于10和非英文标注
            continue
        corp_map[pattern_utt.search(data[0]).group()[1:-1]] = data[1]

In [17]:
aishell_scp = {}
for i in sorted(aishell_scp_raw):
    if i not in corp_map.keys():
        continue
    aishell_scp[i] = aishell_scp_raw[i]

In [18]:
with open('aishell_2.scp', 'w') as f:
    for i in aishell_scp:
        f.write(i+'\t'+aishell_scp[i]+'\n')

In [19]:
with open('aishell_2_trans.txt', 'w') as f:
    for i in corp_map:
        f.write(i+'\t'+corp_map[i]+'\n')

### 筛选一部分到aishell_2.scp.shuff

In [21]:
# !bash tools/random_line.sh aishell_2.scp 2 aishell_2.scp.shuff
! cat aishell_2.scp > aishell_2.scp.shuff

## 合并数据

In [22]:
SLR_wav_scp = {}
SLR_wav_corp = {}
with open("SLR_wav.scp.shuff", "r") as f:
    for line in f:
        data = line.split()
        SLR_wav_scp[data[0]] = data[1]
        SLR_wav_corp[data[0]] = "你好米雅"

In [23]:
aishell_wav_scp = {}
with open("aishell_2.scp.shuff", "r") as f:
    for line in f:
        data = line.split()
        aishell_wav_scp[data[0]] = data[1]
        
aishell_wav_corp = {}       
with open("aishell_2_trans.txt", "r") as f:
    for line in f:
        data = line.split()
        if data[0] in aishell_wav_scp:
            aishell_wav_corp[data[0]] = data[1]

In [24]:
wav_scp = {**SLR_wav_scp,**aishell_wav_scp}
corpus = {**SLR_wav_corp, **aishell_wav_corp}

In [25]:
with open('wav.scp', 'w') as f:
    for i in sorted(wav_scp):
        f.write(i+'\t'+wav_scp[i]+'\n')

In [26]:
with open('corpus.txt', 'w') as f:
    for i in sorted(corpus):
        f.write(i+'\t'+corpus[i]+'\n')

### 生成词典并分词

In [30]:
!sed 's/\s.*$/ 1/' gmm/data/dict/lexicon.txt > seg.dict

In [31]:
#!python seg_word/segmentword.py seg.dict corpus.txt corpus.seg oov_file




vocab_file="seg.dict"
trans_file="corpus.txt"
word_segmented_trans="corpus.seg"

jieba.set_dictionary(vocab_file)
with open(word_segmented_trans, "w") as f:
    for line in open(trans_file):
      key,trans = line.strip().split(None, 1)
      words = jieba.cut(trans, HMM=False) # turn off new word discovery (HMM-based)
      new_line = key + '\t' + " ".join(words)
      f.write(new_line + "\n")



Building prefix dict from /home1/meichaoyang/workspace/git/phone_align/seg.dict ...
Dumping model to file cache /home1/meichaoyang/anaconda3/lib/python3.7/site-packages/jieba/cache/jieba.u9ddf4f6511f6ed6800c70d1cc27817bc.cache
Loading model cost 0.445 seconds.
Prefix dict has been built succesfully.


In [32]:
#!python tools/filter_scp.py corpus.seg wav.scp wav.scp.new

## utt2spk和spk2utt

In [33]:
new_wav_scp_path = "wav.scp"
new_wav_scp = {}
with open(new_wav_scp_path, "r") as f:
    for line in f:
        data = line.split()
        new_wav_scp[data[0]] = data[1]

In [34]:
pattern = re.compile(u'/[^/]+')
str1 = '/home/meichaoyang/dataset/SLR85/dev/SPEECHDATA/wav/SV0255/SV0255_2_00_F0026.wav'
pattern.findall(str1)[-2][1:]

'SV0255'

In [35]:
len(new_wav_scp)

895683

In [36]:
utts = list(new_wav_scp.items())

In [37]:
utt2spk = {}
for utt in utts:
    spk = pattern.findall(utt[1])[-2][1:]
    utt2spk[utt[0]] = spk

In [38]:
spk2utt = {}
for utt in utts:
    spk = pattern.findall(utt[1])[-2][1:]
    if spk not in spk2utt:
        spk2utt[spk] = []
    spk2utt[spk].append(utt[0])

In [39]:
with open('utt2spk', 'w') as f:
    for i in sorted(utt2spk):
        f.write(i+'\t'+utt2spk[i]+'\n')

In [40]:
len(utts)

895683

#### 根据utt2spk生成spk2utt

In [41]:
!gmm/utils/utt2spk_to_spk2utt.pl utt2spk > spk2utt

#### spk2utt，utt2spk，text(corpus.seg)，wav.scp(wav.scp.new)复制到train_{mfcc,fbank}

In [42]:
!rm -rf train_mfcc train_fbank
!mkdir train_mfcc train_fbank
!cp spk2utt utt2spk corpus.seg wav.scp train_mfcc

In [43]:
!mv train_mfcc/corpus.seg train_mfcc/text 
#!mv train_mfcc/wav.scp.new train_mfcc/wav.scp
!cd train_mfcc &&cp spk2utt utt2spk text wav.scp ../train_fbank

#### 特征提取`cd ext_feats && sh ext_{mfcc,fbank}.sh {train_fbank,train_mfcc,dev}`，需注意修改conf下mfcc，fbank参数

In [ ]:
!cd ext_feats && sh ext_mfcc.sh ../train_mfcc && utils/fix_data_dir.sh ../train_mfcc

steps/make_mfcc.sh --nj 48 --cmd run.pl ../train_mfcc ../train_mfcc/log ../train_mfcc/_mfcc
utils/validate_data_dir.sh: Successfully validated data-directory ../train_mfcc
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
Succeeded creating MFCC features for train_mfcc
steps/compute_cmvn_stats.sh ../train_mfcc ../train_mfcc/cmvn_log ../train_mfcc/_cmvn


In [45]:
!cd ext_feats && sh ext_fbank.sh ../train_fbank && utils/fix_data_dir.sh ../train_fbank

Succeeded creating filterbank features for train_fbank
steps/compute_cmvn_stats.sh ../train_fbank ../train_fbank/cmvn_log ../train_fbank/_cmvn
Succeeded creating CMVN stats for train_fbank
fix_data_dir.sh: kept all 895683 utterances.
fix_data_dir.sh: old files are kept in ../train_fbank/.backup


In [65]:
!ln -s ../../train_mfcc gmm/data/train
!ln -s ../../train_fbank chain/data/train_fbank
!ln -s ../../gmm/data/train/ chain/data/train_mfcc

ln: failed to create symbolic link 'gmm/data/train/train_mfcc': File exists
ln: failed to create symbolic link 'chain/data/train_mfcc/train': File exists


In [47]:
!cd gmm && make prepare_lang

utils/prepare_lang.sh --position-dependent-phones false `pwd`/data/dict "SPOKEN_NOISE" `pwd`/data/local/lang `pwd`/data/lang
utils/prepare_lang.sh --position-dependent-phones false /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict SPOKEN_NOISE /home1/meichaoyang/workspace/git/phone_align/gmm/data/local/lang /home1/meichaoyang/workspace/git/phone_align/gmm/data/lang
Checking /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict/silence_phones.txt ...
--> reading /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict/silence_phones.txt is OK

Checking /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict/optional_silence.txt ...
--> reading /home1/meichaoyang/workspace/git/phone_align/gmm/data/dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text 

--> /home1/meichaoyang/workspace/git/phone_align/gmm/data/lang/L.fst is olabel sorted
--> /home1/meichaoyang/workspace/git/phone_align/gmm/data/lang/L_disambig.fst is olabel sorted
--> SUCCESS [validating lang directory /home1/meichaoyang/workspace/git/phone_align/gmm/data/lang]


In [48]:
!cd gmm &&make -f Makefile gmm  | tee gmm.log

local/train_gmm.sh `pwd`/data/train `pwd`/data/cv
steps/train_mono.sh --boost-silence 1.25 --nj 24 --cmd run.pl /home1/meichaoyang/workspace/git/phone_align/gmm/data/train data/lang exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 1

steps/align_si.sh: feature type is lda
steps/align_si.sh: aligning data in /home1/meichaoyang/workspace/git/phone_align/gmm/data/train using model from exp/tri2b, putting alignments in exp/tri2b_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl data/lang exp/tri2b_ali
steps/diagnostic/analyze_alignments.sh: see stats in exp/tri2b_ali/log/analyze_alignments.log
steps/align_si.sh: done aligning data.
steps/train_sat.sh --cmd run.pl 15000 180000 /home1/meichaoyang/workspace/git/phone_align/gmm/data/train data/lang exp/tri2b_ali exp/tri3b
steps/train_sat.sh: feature type is lda
steps/train_sat.sh: obtaining initial fMLLR transforms since not present in exp/tri2b_ali
steps/train_sat.sh: Accumulating tree stats
steps/train_sat.sh: Getting questions for tree clustering.
steps/train_sat.sh: Building the tree
steps/train_sat.sh: Initializing the model
steps/train_sat.sh: Converting alignments from exp/tri2b_ali to use current tree
steps/train_sat.sh: Compiling graphs of transcripts
Pass 1

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2 && cd chain && make chain |  tee chain.log

local/train_chain.sh exp/chain/tdnn_attend
local/train_chain.sh exp/chain/tdnn_attend
steps/align_fmllr_lats.sh --nj 24 --cmd run.pl data/train_mfcc data/lang exp/tri4b exp/tri4b_lats
steps/align_fmllr_lats.sh: feature type is lda
steps/align_fmllr_lats.sh: compiling training graphs
steps/align_fmllr_lats.sh: aligning data in data/train_mfcc using exp/tri4b/final.alimdl and speaker-independent features.
steps/align_fmllr_lats.sh: computing fMLLR transforms
steps/align_fmllr_lats.sh: generating lattices containing alternate pronunciations.
steps/align_fmllr_lats.sh: done generating lattices from training transcripts.
894 warnings in exp/tri4b_lats/log/fmllr.*.log
691 warnings in exp/tri4b_lats/log/generate_lattices.*.log
3809 warnings in exp/tri4b_lats/log/align_pass1.*.log
steps/nnet3/chain/build_tree.sh --frame-subsampling-factor 3 --leftmost-questions-truncate -1 --context-opts --context-width=2 --central-position=1 --cmd run.pl 7000 data/train_mfcc data/lang_new exp/tri4b_ali exp/ch

2020-03-18 11:54:52,690 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 3/1265    Epoch: 0.00/4.0 (0.1% complete)    lr: 0.000997    
2020-03-18 11:55:23,756 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 4/1265    Epoch: 0.01/4.0 (0.2% complete)    lr: 0.000996    
2020-03-18 11:55:54,593 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 5/1265    Epoch: 0.01/4.0 (0.2% complete)    lr: 0.000995    
2020-03-18 11:56:25,694 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 6/1265    Epoch: 0.01/4.0 (0.2% complete)    lr: 0.000995    
2020-03-18 11:56:57,040 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 7/1265    Epoch: 0.01/4.0 (0.3% complete)    lr: 0.000994    
2020-03-18 11:57:31,454 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 8/1265    Epoch: 0.01/4.0 (0.3% complete)    lr: 0.000993    
2020-03-18 11:58:06,263 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 9/1265    Epoch: 0.01/4.0 (0.4% complete)    lr: 0.000992    
2020-03-18 11

2020-03-18 12:28:07,390 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 61/1265    Epoch: 0.10/4.0 (2.4% complete)    lr: 0.000946    
2020-03-18 12:28:41,985 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 62/1265    Epoch: 0.10/4.0 (2.4% complete)    lr: 0.000945    
2020-03-18 12:29:16,663 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 63/1265    Epoch: 0.10/4.0 (2.5% complete)    lr: 0.000944    
2020-03-18 12:29:50,578 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 64/1265    Epoch: 0.10/4.0 (2.5% complete)    lr: 0.000943    
2020-03-18 12:30:24,741 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 65/1265    Epoch: 0.10/4.0 (2.6% complete)    lr: 0.000943    
2020-03-18 12:30:59,465 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 66/1265    Epoch: 0.10/4.0 (2.6% complete)    lr: 0.000942    
2020-03-18 12:31:32,764 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 67/1265    Epoch: 0.11/4.0 (2.6% complete)    lr: 0.000941    
2020-0

2020-03-18 13:01:25,615 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 119/1265    Epoch: 0.19/4.0 (4.7% complete)    lr: 0.000897    
2020-03-18 13:02:00,530 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 120/1265    Epoch: 0.19/4.0 (4.7% complete)    lr: 0.000897    
2020-03-18 13:02:36,300 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 121/1265    Epoch: 0.19/4.0 (4.8% complete)    lr: 0.000896    
2020-03-18 13:03:10,929 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 122/1265    Epoch: 0.19/4.0 (4.8% complete)    lr: 0.000895    
2020-03-18 13:03:45,535 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 123/1265    Epoch: 0.19/4.0 (4.9% complete)    lr: 0.000894    
2020-03-18 13:04:20,119 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 124/1265    Epoch: 0.20/4.0 (4.9% complete)    lr: 0.000893    
2020-03-18 13:04:54,819 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 125/1265    Epoch: 0.20/4.0 (4.9% complete)    lr: 0.000893    

2020-03-18 13:34:42,186 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 177/1265    Epoch: 0.28/4.0 (7.0% complete)    lr: 0.000851    
2020-03-18 13:35:17,129 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 178/1265    Epoch: 0.28/4.0 (7.0% complete)    lr: 0.000851    
2020-03-18 13:35:51,097 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 179/1265    Epoch: 0.28/4.0 (7.1% complete)    lr: 0.000850    
2020-03-18 13:36:25,725 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 180/1265    Epoch: 0.28/4.0 (7.1% complete)    lr: 0.000849    
2020-03-18 13:37:00,366 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 181/1265    Epoch: 0.29/4.0 (7.1% complete)    lr: 0.000848    
2020-03-18 13:37:34,933 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 182/1265    Epoch: 0.29/4.0 (7.2% complete)    lr: 0.000847    
2020-03-18 13:38:10,040 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 183/1265    Epoch: 0.29/4.0 (7.2% complete)    lr: 0.000847    

...

2020-03-18 18:11:31,389 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 635/1265    Epoch: 1.51/4.0 (37.6% complete)    lr: 0.000841    
2020-03-18 18:12:20,341 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 636/1265    Epoch: 1.51/4.0 (37.7% complete)    lr: 0.000839    
2020-03-18 18:13:08,491 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 637/1265    Epoch: 1.51/4.0 (37.8% complete)    lr: 0.000838    
2020-03-18 18:13:53,716 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 638/1265    Epoch: 1.52/4.0 (37.9% complete)    lr: 0.000836    
2020-03-18 18:14:31,411 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 639/1265    Epoch: 1.52/4.0 (38.0% complete)    lr: 0.000835    
2020-03-18 18:15:05,883 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 640/1265    Epoch: 1.52/4.0 (38.0% complete)    lr: 0.000833    
2020-03-18 18:15:51,833 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 641/1265    Epoch: 1.52/4.0 (38.1% complete)    lr: 0.000

2020-03-18 18:52:49,461 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 692/1265    Epoch: 1.69/4.0 (42.1% complete)    lr: 0.000758    
2020-03-18 18:53:28,239 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 693/1265    Epoch: 1.69/4.0 (42.2% complete)    lr: 0.000757    
2020-03-18 18:54:21,205 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 694/1265    Epoch: 1.69/4.0 (42.3% complete)    lr: 0.000755    
2020-03-18 18:55:11,474 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 695/1265    Epoch: 1.70/4.0 (42.4% complete)    lr: 0.000754    
2020-03-18 18:55:58,540 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 696/1265    Epoch: 1.70/4.0 (42.5% complete)    lr: 0.000752    
2020-03-18 18:56:37,364 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 697/1265    Epoch: 1.70/4.0 (42.5% complete)    lr: 0.000751    
2020-03-18 18:57:15,441 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 698/1265    Epoch: 1.70/4.0 (42.6% complete)    lr: 0.000

2020-03-18 20:15:48,357 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 805/1265    Epoch: 2.04/4.0 (51.1% complete)    lr: 0.000617    
2020-03-18 20:16:25,333 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 806/1265    Epoch: 2.05/4.0 (51.1% complete)    lr: 0.000616    
2020-03-18 20:17:06,827 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 807/1265    Epoch: 2.05/4.0 (51.2% complete)    lr: 0.000615    
2020-03-18 20:17:49,162 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 808/1265    Epoch: 2.05/4.0 (51.3% complete)    lr: 0.000614    
2020-03-18 20:18:51,562 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 809/1265    Epoch: 2.06/4.0 (51.4% complete)    lr: 0.000613    
2020-03-18 20:19:50,273 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 810/1265    Epoch: 2.06/4.0 (51.5% complete)    lr: 0.000612    
2020-03-18 20:20:48,680 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 811/1265    Epoch: 2.06/4.0 (51.5% complete)    lr: 0.000

2020-03-18 20:59:28,180 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 862/1265    Epoch: 2.22/4.0 (55.6% complete)    lr: 0.000556    
2020-03-18 21:00:15,463 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 863/1265    Epoch: 2.23/4.0 (55.6% complete)    lr: 0.000555    
2020-03-18 21:01:06,993 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 864/1265    Epoch: 2.23/4.0 (55.7% complete)    lr: 0.000554    
2020-03-18 21:01:59,357 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 865/1265    Epoch: 2.23/4.0 (55.8% complete)    lr: 0.000553    
2020-03-18 21:02:57,680 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 866/1265    Epoch: 2.24/4.0 (55.9% complete)    lr: 0.000552    
2020-03-18 21:03:54,273 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 867/1265    Epoch: 2.24/4.0 (56.0% complete)    lr: 0.000551    
2020-03-18 21:04:50,321 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 868/1265    Epoch: 2.24/4.0 (56.0% complete)    lr: 0.000

2020-03-18 21:45:04,653 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 919/1265    Epoch: 2.40/4.0 (60.1% complete)    lr: 0.000502    
2020-03-18 21:45:50,156 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 920/1265    Epoch: 2.41/4.0 (60.2% complete)    lr: 0.000501    
2020-03-18 21:46:38,356 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 921/1265    Epoch: 2.41/4.0 (60.2% complete)    lr: 0.000500    
2020-03-18 21:47:17,395 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 922/1265    Epoch: 2.41/4.0 (60.3% complete)    lr: 0.000499    
2020-03-18 21:47:55,770 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 923/1265    Epoch: 2.42/4.0 (60.4% complete)    lr: 0.000498    
2020-03-18 21:48:32,465 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 924/1265    Epoch: 2.42/4.0 (60.5% complete)    lr: 0.000497    
2020-03-18 21:49:10,186 [steps/nnet3/chain/train.py:509 - train - INFO ] Iter: 925/1265    Epoch: 2.42/4.0 (60.5% complete)    lr: 0.000

In [69]:
!export CUDA_VISIBLE_DEVICES=0,1,2 && cd chain &&./steps/nnet3/align.sh --nj 3 --use_gpu true ../train_fbank ../gmm/data/lang exp/chain/tdnn_attend align

./steps/nnet3/align.sh --nj 3 --use_gpu true ../train_fbank ../gmm/data/lang exp/chain/tdnn_attend align
./steps/nnet3/align.sh: feature type is raw
./steps/nnet3/align.sh: aligning data in ../train_fbank using model from exp/chain/tdnn_attend, putting alignments in align
./steps/nnet3/align.sh: frame-subsampling-factor is not 1 (so likely a chain system),
...  but the scale opts are the defaults.  You probably want
--scale-opts '--transition-scale=1.0 --acoustic-scale=1.0 --self-loop-scale=1.0'
steps/diagnostic/analyze_alignments.sh --cmd run.pl ../gmm/data/lang align
steps/diagnostic/analyze_alignments.sh: see stats in align/log/analyze_alignments.log
./steps/nnet3/align.sh: done aligning data.


In [74]:
!cd chain && ./steps/get_train_ctm.sh  data/train_fbank data/lang align

./steps/get_train_ctm.sh data/train_fbank data/lang align


In [47]:
#!sed 's/[^\s]*\s//' corpus.seg > corpus_lm.txt
!ngram-count -text corpus_lm.txt -order 1 -unk -map-unk "<UNK>" -interpolate -lm corpus.lm

In [48]:
!cd chain/ && make make_graph

local/make_graph.sh /home1/meichaoyang/Workspace/wake_up/corpus.lm `pwd`/data/lang exp/chain/tdnn_attend `pwd`/data/dict
gzip: /home1/meichaoyang/Workspace/wake_up/corpus.lm: No such file or directory
Makefile:18: recipe for target 'make_graph' failed
make: *** [make_graph] Error 1


In [50]:
!du -sh chain/exp

du: cannot access 'chain/exp': No such file or directory
